In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

from SPIworkflow.__init__ import *
from SPIworkflow.constants import *

In [2]:
def Kepler_r(M_star, P_orb):
    """Computes the orbital radius of a planet (=semimajor axis, in units of au)
       orbiting a star of mass M_star (in solar masses) and orbital period, P_orb (in days)
    """
    r_orb = (G * M_star*M_sun)**(1/3) * (P_orb*day/2/np.pi)**(2/3)/au
    return r_orb

In [3]:
print(Kepler_r(1.,365.4))

1.0003658870652499


In [4]:
def Kepler_P(M_star, r_orb):
    """Computes the orbital period (P_orb) of a planet (in days) of a planet
       orbiting a star of mass M_star (in solar masses) and semi-major axis (r_orb), in au.
    """
    
    P_orb = 2*np.pi * (G * M_star*M_sun)**(-1/2) * (r_orb*au)**(3/2)/day
    return P_orb


In [5]:
print(Kepler_P(1.,1.))

365.1995489803746


In [8]:
def Rp_Zeng(Mp = 1.0):
    """ Computes the planet radius using the mass-radius relationship in 
        in Zeng et al. (PNAS, 2019): 
        https://www.pnas.org/content/116/20/9723
        R [R_earth] = f * (M/M_earth)^(1/3.7)  (for rocky planets, f=1.)
        
        OUTPUT: Rp - planet radius, in units of Earth radius 
        INPUT:  Mp - planet mass, in units of Earth mass
        We assume f = 1 (Earth-like rocky planet)
    """
    f = 1.0
    Rp = f * Mp**(1/3.7)
    return Rp

In [11]:
print(Rp_Zeng(8.0))

1.754197044080575


In [13]:
def get_open_field_values(B_star, d_orb, R_star, vcorot, v_sw):
    if open_field: 
        # Open Parker Spiral - Falls down with distances as R^(-2) rather than R^(-3) as in the dipole case
        B_r = B_star * (d_orb/R_star)**(-2) # Stellar wind B-field at (R/R_star), Eqn 20 Turnpenney 2018
        B_phi = B_r * v_corot/v_sw # Azimuthal field (Eqn 21 Turnpenney 2018)
        B_sw = np.sqrt(B_r**2 + B_phi**2) # Total stellar wind B-field at planet orbital distance

        # Eq. 23 of Turnpenney 2018 -  First term of RHS
        B_ang = np.arctan(B_phi/B_r) # Angle the B-field makes with the radial direction

        # Angle between the stellar wind magnetic field and the impinging plasma velocity
        # Eqn 23 in Turnpenney 2018. It's also Eq. 13 in Zarka 2007
        theta = np.absolute(B_ang - v_rel_angle) 

        # theta is the angle between the B_sw (the insterstellar magnetic field), and the
        # incident stellar wind velocity.  See Fig. 1 in Turnpenney+2018
        #
        geom_f = (np.sin(theta))**2 # Geometric factor in efficiency 
    else:
        # Closed, dipolar configuration - It falls with distance as R^(-3)
        # B_star - magnetic field at the magnetic equator on the stellar surface
        # 
        phi = 0. # azimuth, measured from the North magnetic pole of the star (in degrees)
        phi *= np.pi/180. # to radians

        B_r   = -2 * B_star * (d_orb/R_star)**(-3) * np.cos(phi) # Radial component of the dipole magnetic field of the stellar wind as f(distance to star)
        B_phi = - B_star * (d_orb/R_star)**(-3) * np.sin(phi) # Azimuthal component of the dipole magnetic field 
        B_sw = np.sqrt(B_r**2 + B_phi**2) # Total dipole magnetic field 

        geom_f = 1.0 # Geometric factor. 1 for closed dipole configuration, different for the open field configuration

        return B_r, B_phi, B_sw, B_ang, theta, geom_f

In [17]:
print(np.tan(np.pi/2.))

1.633123935319537e+16


In [19]:
print(np.arctan(1.633123935319537e+16))

1.5707963267948966


In [21]:
print(0.5*m_p)

8.3631095e-25
